In [ ]:
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import glob
from autoreject import AutoReject, get_rejection_threshold

#### Read Raw Data

In [ ]:
# Read BrainVision EEG data into 'raw' object
data_folder = './eeg_data/rise'
subj = 'Rise0003'
file_path = os.path.join(data_folder, subj+'.vhdr')
raw = mne.io.read_raw_brainvision(file_path, verbose=False)
raw

#### Rename Channels

In [ ]:
# Replace all channels with proper channel names instead of numbers
channel_names_old = raw.ch_names
channel_names_new = ['Fp1','Fz','F3','F7','FT9','FC5','FC1','C3','T7','TP9','CP5','CP1','Pz','P3','P7','O1','Oz','O2','P4','P8','TP10','CP6',
                        'CP2','C4','T8','FT10','FC6','FC2','F4','F8','Fp2', 'AF7','AF3','AFz','F1','F5','FT7','FC3','C1','C5','TP7','CP3','P1','P5',
                        'PO7','PO3','POz','PO4','PO8','P6','P2','CPz','CP4','TP8','C6','C2','FC4','FT8','F6','AF8','AF4','F2','FCz', 'Cz']
channel_dict = dict(zip(channel_names_old, channel_names_new))
mne.rename_channels(raw.info, mapping=channel_dict)

#### Set Montage 

In [ ]:
easycap_montage = mne.channels.make_standard_montage('easycap-M1')

# Use the preloaded montage
raw.set_montage(easycap_montage)
fig = raw.plot_sensors(show_names=True)

raw.info

#### Get Events

In [ ]:
# For subject 1, 2 and 3
events_from_annot, event_dict = mne.events_from_annotations(raw)

def fix_events(events):

    # Get indices of R11 events with event code 1011 or New Segment indices with event code 99999 and delete them
    useless_events = list(filter(lambda i: events[:, 2][i] == 99999 or events[:, 2][i] == 1011 or events[:, 2][i] == 2 or events[:, 2][i] == 6, range(len(events[:, 2]))))
    events = np.delete(events, useless_events, 0)

    for i in range(len(events[:, 2])):
        if events[:, 2][i] == 1004 or events[:, 2][i] == 1008:
            events[:, 2][i] = 1001
        elif  events[:, 2][i] == 1012:
            events[:, 2][i] = 1002
        elif events[:, 2][i] == 1024:
            events[:, 2][i] = 1003
        elif events[:, 2][i] == 1028:
            events[:, 2][i] = 1004

    return events

fixed_events = fix_events(events_from_annot)

s_events = fixed_events
print(len(s_events))

In [ ]:
# For subjects 4 onwards
events_from_annot, event_dict = mne.events_from_annotations(raw)

# Function to fix events
def fix_events(events):

    # Get indices of R11 events with event code 1011 or New Segment indices with event code 99999 and delete them
    useless_events = list(filter(lambda i: events[:, 2][i] == 99999 or events[:, 2][i] == 1011, range(len(events[:, 2]))))
    events = np.delete(events, useless_events, 0)

    # Get indices of consective equal events and keep only the 1st one and delete the others
    consecutive_equal_events = list(filter(lambda i: events[:, 2][i] == events[:, 2][i+1], range(len(events[:, 2])-1)))
    consecutive_equal_events = [index+1 for index in consecutive_equal_events]
    events = np.delete(events, consecutive_equal_events, 0)

    return events

def get_s_events(events):
    # If experiment starts with an S event with event code 2, delete it
    if events[:, 2][0] == 2:
        events = np.delete(events, 0, 0)

    # Change S event so it takes event code from R events (for e.g.- if R3 is followed by S2, we will change it to S3)
    for i in range(0, len(events), 2):
        events[i+1][2] = events[i][2]

    # Extract S events into different array
    s_events = events[1::2]

    return s_events


fixed_events = fix_events(events_from_annot)
s_events = get_s_events(fixed_events)

#### Create Events Channel in Raw Data

In [ ]:
# Create event channel
raw.load_data()
stim_data = np.zeros((1, len(raw.times)))

# Add stimulus channel in 'raw' object's info class
info = mne.create_info(['STI'], raw.info['sfreq'], ['stim'])
stim_raw = mne.io.RawArray(stim_data, info)
raw.add_channels([stim_raw], force_update_info=True)

# Add events extracted from annotations to the stimulus channel
raw.add_events(s_events, stim_channel='STI')
print(raw.info)

#### Crop Raw Data to Remove Useless Parts

In [ ]:
# Get event times
first_event_time = mne.find_events(raw)[0][0]
last_event_time = mne.find_events(raw)[-1][0]
print(first_event_time, ' ', last_event_time)

# Remove useless parts of raw EEG
epoch_limits = [-0.1, 0.6]
part_to_remove_from_beginning = (first_event_time - abs(epoch_limits[0]*500))/1000
part_to_remove_from_end = (last_event_time + abs(epoch_limits[1]*5000))/1000
raw.crop(part_to_remove_from_beginning, part_to_remove_from_end)

#### Filtering

In [ ]:
# Soft bandpass Butterworth filter 
iir_params = dict(  order=2, 
                    ftype='butter', 
                    output='sos'
                )
iir_params = mne.filter.construct_iir_filter(   iir_params, 
                                                f_pass=[0.1, 30], 
                                                f_stop=None, 
                                                sfreq=1000, 
                                                btype='bandpass', 
                                                return_copy=False
                                            )
raw.filter(0.1, 30, method='iir', iir_params=iir_params)

# Notch filter
raw.notch_filter(   freqs=np.arange(50, 251, 50), 
                    method='fir', 
                    fir_design='firwin2'
                )

#### Add Reference Channel (Cz)

In [ ]:
mne.add_reference_channels(raw, 'Cz', copy=False)

#### Re-reference electrodes

In [ ]:
mne.set_eeg_reference(raw, ref_channels='average', projection=True)
raw.apply_proj()

#### Reset Montage after adding Cz

In [ ]:
raw.set_montage(easycap_montage)

#### Epoching

In [ ]:
# Don't baseline correct before ICA
epochs = mne.Epochs(raw, s_events, tmin=-0.1, tmax=0.6, preload=True, baseline=None)

#### Autoreject on All Epochs

In [ ]:
ar = AutoReject(n_interpolate=[1, 2, 3, 4], random_state=11, n_jobs=1, verbose=True)
ar.fit(epochs) 
epochs_ar, reject_log = ar.transform(epochs, return_log=True)

#### ICA

In [ ]:
ica = mne.preprocessing.ICA(random_state=99)
ica.fit(epochs[~reject_log.bad_epochs])

# Find which ICs match the EOG pattern
eog_indices, eog_scores = ica.find_bads_eog(epochs[~reject_log.bad_epochs], ch_name='Fp1')
print(f'**************** Automatically found EOG artifact ICA components: {eog_indices} ****************')

# # Find which ICs match the EMG pattern
# muscle_idx_auto, scores = ica.find_bads_muscle(epochs[~reject_log.bad_epochs])
# print(f'**************** Automatically found muscle artifact ICA components: {muscle_idx_auto} ****************')

ica.exclude = eog_indices

ica.plot_overlay(epochs.average(), exclude=ica.exclude)
ica.apply(epochs, exclude=ica.exclude)

#### Baseline Correction

In [ ]:
epochs.apply_baseline(baseline=(-0.1, 0))

#### Autoreject on All Epochs

In [ ]:
ar = AutoReject(n_interpolate=[1, 2, 3, 4], random_state=11, n_jobs=1, verbose=True)
ar.fit(epochs) 
epochs_ar, reject_log = ar.transform(epochs, return_log=True)

#### Save epochs to file

In [ ]:
epochs_ar.save('./analysis/'+subj+'-epo.fif', overwrite=True)

#### Create Evoked objects and save to file

In [ ]:
# Create Evoked object from epochs (an Evoked object contains the average data over all epochs)
evoked_standard = epochs_ar['1001'].average()
evoked_neutral = epochs_ar['1002'].average()
evoked_rise = epochs_ar['1003'].average()
evoked_fall = epochs_ar['1004'].average()

mne.write_evokeds('./analysis/'+subj+'-ave.fif', [evoked_standard, evoked_neutral, evoked_rise, evoked_fall], overwrite=True)

evokeds = dict(standard=evoked_standard, neutral=evoked_neutral, rise=evoked_rise, fall=evoked_fall)

#### Plot ERPs

In [ ]:
def plot_channel_by_condition(channels=[]):
    # Create PDF file in which to save all plots
    with matplotlib.backends.backend_pdf.PdfPages('./analysis/'+subj+'-plots.pdf') as pdf:
    
        for channel in channels:
            fig = mne.viz.plot_compare_evokeds(evokeds, picks=channel, combine=None, time_unit='ms', ylim=dict(eeg=[-10, 10]), invert_y=True,
                                            colors=dict(standard='black', neutral='red', rise='blue', fall='green'), 
                                            styles={'standard': {'linewidth': 1}, 'neutral': {'linewidth': 1}, 'rise': {'linewidth': 1}, 'fall': {'linewidth': 1}})
            # Save plot to PDF
            pdf.savefig(fig[0])
            plt.close()

# Plot channels Fz, Pz, Oz, AFz, POz, CPz, FCz, Cz
plot_channel_by_condition(channels=['Fz', 'Pz', 'Oz', 'AFz', 'POz', 'CPz', 'FCz', 'Cz'])